In [25]:
%load_ext autoreload
%autoreload 2

from read_excel import *
from compute_state import *
from collections import namedtuple

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Example usage:
file_path = 'ordenes.xlsx'  # Set the filename to 'ordenes.xlsx'
df = read_excel_to_dataframe(file_path)
df = merge_date_time_columns(df)
print(df.loc[0])

Operacion           Suscripción Desde Cuenta Balanz
Estado                                    Ejecutada
id Orden                                   41387004
Ticker                                        BCMMA
Moneda                                        Pesos
Fecha                           2024-04-17 14:15:11
Cantidad                                       -1.0
Precio                                         -1.0
Monto                                     314704.13
Precio Operado                             6.300322
Cantidad Operada                       49950.483483
Name: 0, dtype: object


## Example usage:
# Create a Balance tuple
balance = Balance(ARS=0, USD=0)

# Create a State tuple containing the Balance tuple
current_state = State(balance=balance)

# Iterate through all rows of the DataFrame
for index, row in df.iterrows():
    # Call the update_state function for each row
    next_state = update_state(row, current_state)
    
    # Update the current state to the next state
    current_state = next_state

current_state